In [ ]:
import pandas as pd
from google.colab import files

def load_csv_file():
    print("Please upload your CSV file.")
    uploaded = files.upload()

    if not uploaded:
        print("No file uploaded.")
        return None

    filename = next(iter(uploaded))
    print(f"Loading '{filename}' into DataFrame.")
    df = pd.read_csv(filename)

    return df

# Load the uploaded CSV file into a DataFrame
data = load_csv_file()

# Check if the data is loaded successfully
if data is not None:
    print("Data loaded successfully.")
    print(data.head())  # Show the first few rows of the DataFrame
else:
    print("Data loading failed.")

Please upload your CSV file.


Saving wiki_movie_plots_deduped.csv to wiki_movie_plots_deduped.csv
Loading 'wiki_movie_plots_deduped.csv' into DataFrame.
Data loaded successfully.
   Release Year                             Title Origin/Ethnicity  \
0          1901            Kansas Saloon Smashers         American   
1          1901     Love by the Light of the Moon         American   
2          1901           The Martyred Presidents         American   
3          1901  Terrible Teddy, the Grizzly King         American   
4          1902            Jack and the Beanstalk         American   

                             Director Cast    Genre  \
0                             Unknown  NaN  unknown   
1                             Unknown  NaN  unknown   
2                             Unknown  NaN  unknown   
3                             Unknown  NaN  unknown   
4  George S. Fleming, Edwin S. Porter  NaN  unknown   

                                           Wiki Page  \
0  https://en.wikipedia.org/wiki/Kansas_Sal

Explantion
we loaded the csv file using the upload button

In [ ]:
import pandas as pd

def process_data(filename, output_filename):
    # Load only the 'Title' and 'Plot' columns from the CSV file
    df = pd.read_csv(filename, usecols=['Title', 'Plot'])

    # Here you can apply any function to the dataframe if needed
    # For example, you could do some data cleaning or processing
    # processed_df = your_function(df)

    # For this example, we'll just use the original dataframe
    processed_df = df

    # Save the processed data to a new CSV file
    processed_df.to_csv(output_filename, index=False)
    print(f"Data processed and saved to '{output_filename}'.")

# Specify the name of the already uploaded file and the output file
input_filename = '/content/wiki_movie_plots_deduped.csv'
output_filename = '/content/processed_movies.csv'

# Process the data and save it to another file
process_data(input_filename, output_filename)

Data processed and saved to '/content/processed_movies.csv'.


Explanation:
we have removed the extra columns from the dataset and kept "title" and "plot" columns

In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 632.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

we have installed the sentence transformer libraries

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch

# Load the sentence transformer model
model = SentenceTransformer('nq-distilbert-base-v1')

# Load your data here
# For the sake of this example, let's assume it's already loaded in a variable `data`
data = pd.read_csv('/content/processed_movies.csv')

# Combine the Title and Plot for indexing
corpus = [f"{row['Title']} {row['Plot']}" for _, row in data.iterrows()]

# Encode the corpus using the model
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

# Function to search the corpus
def search(query: str, top_k: int = 5):
    # Encode the query
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Compute the similarities between the query and the corpus
    similarities = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]

    # Find the top_k most similar entries
    top_results = torch.topk(similarities, k=top_k)

    print("Query:", query)
    print("\nTop 5 most similar movies in the corpus:")
    for score, idx in zip(top_results[0], top_results[1]):
        print(data.iloc[idx.item()]['Title'], "(Score: {:.4f})".format(score))

# Get a search query from the user
user_query = input("Enter your movie plot query: ")
search(user_query)

Enter your movie plot query: Documentaries showcasing indigenous peoples' survival and daily life in Arctic regions
Query: Documentaries showcasing indigenous peoples' survival and daily life in Arctic regions

Top 5 most similar movies in the corpus:
Nanook of the North (Score: 0.4522)
Kizhakkunarum Pakshi (Score: 0.4347)
The 11th Hour (Score: 0.4232)
O Pioneers! (Score: 0.4131)
Being Caribou (Score: 0.4048)


Let us calculate the values of @Recall and @MRR now.
Recall@k = true positives@k + true negatives@k
1. Documentaries showcasing indigenous pepoles survival and daily life in arctic regions
The top 5 results for this query are : Nanook of the north,Kizhakunarum Pakshi,The 11th Hour,O Pioneers!,Being Caribou
out of these five results, Nanook of the north - RELEVANT, Kizhakkunarum Pakshi - IRRELEVANT, The 11th hour - IRRELEVANT, O Pioneers! - Irrelevant, Being Caribou - RELEVANT
Recall@1 = 1/(1+1)= 0.5


The metric is useful when we our system to return the best relevant item and want that item to be at higher position.
The top 5 results for this query are : Nanook of the north,Kizhakunarum Pakshi,The 11th Hour,O Pioneers!,Being Caribou
For this query, the reciporcal rank is 1/1 and MRR=1 (as the first correct item is at position)

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch

# Load the sentence transformer model
model = SentenceTransformer('nq-distilbert-base-v1')

# Load your data here
# For the sake of this example, let's assume it's already loaded in a variable `data`
data = pd.read_csv('/content/processed_movies.csv')

# Combine the Title and Plot for indexing
corpus = [f"{row['Title']} {row['Plot']}" for _, row in data.iterrows()]

# Encode the corpus using the model
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

# Function to search the corpus
def search(query: str, top_k: int = 5):
    # Encode the query
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Compute the similarities between the query and the corpus
    similarities = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]

    # Find the top_k most similar entries
    top_results = torch.topk(similarities, k=top_k)

    print("Query:", query)
    print("\nTop 5 most similar movies in the corpus:")
    for score, idx in zip(top_results[0], top_results[1]):
        print(data.iloc[idx.item()]['Title'], "(Score: {:.4f})".format(score))

# Get a search query from the user
user_query = input("Enter your movie plot query: ")
search(user_query)

Enter your movie plot query: Western romance
Query: Western romance

Top 5 most similar movies in the corpus:
A Saloon Wet with Beautiful Women (Score: 0.4185)
Lal Dupatta Malmal Ka (Score: 0.3901)
Goodnight, Sweetheart (Score: 0.3745)
Book of Love (Score: 0.3481)
Modern Romance (Score: 0.3409)


Let us calculate the values of @Recall and @MRR now. Recall@k = true positives@k + true negatives@k

Documentaries showcasing indigenous pepoles survival and daily life in arctic regions The top 5 results for this query are : A Saloon Wet With Beautiful Women, Lal Dupatta Malmal ka, goodnight,sweetheart, book of love, modern Romance. Out of five results A saloon wet with beautiful women - Relevant, Lal Duppatta Malmal ka - irrelevant, goodnight,sweetheart - irrelevant, book of love - relevant, Modern romance - Relevant
recall@1 - 1/(1+1) = 0.5

The metric is useful when we our system to return the best relevant item and want that item to be at higher position.
The top 5 results for this query are :A Saloon Wet With Beautiful Women, Lal Dupatta Malmal ka, goodnight,sweetheart, book of love, modern Romance
For this query, the reciporcal rank is 1/1 and MRR=1 (as the first correct item is at position)

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch

# Load the sentence transformer model
model = SentenceTransformer('nq-distilbert-base-v1')

# Load your data here
# For the sake of this example, let's assume it's already loaded in a variable `data`
data = pd.read_csv('/content/processed_movies.csv')

# Combine the Title and Plot for indexing
corpus = [f"{row['Title']} {row['Plot']}" for _, row in data.iterrows()]

# Encode the corpus using the model
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

# Function to search the corpus
def search(query: str, top_k: int = 5):
    # Encode the query
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Compute the similarities between the query and the corpus
    similarities = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]

    # Find the top_k most similar entries
    top_results = torch.topk(similarities, k=top_k)

    print("Query:", query)
    print("\nTop 5 most similar movies in the corpus:")
    for score, idx in zip(top_results[0], top_results[1]):
        print(data.iloc[idx.item()]['Title'], "(Score: {:.4f})".format(score))

# Get a search query from the user
user_query = input("Enter your movie plot query: ")
search(user_query)

Enter your movie plot query: Silent film about a Parisian star moving to Egypt, leaving her husband for a baron, and later reconciling after finding her family in poverty in Cairo.
Query: Silent film about a Parisian star moving to Egypt, leaving her husband for a baron, and later reconciling after finding her family in poverty in Cairo.

Top 5 most similar movies in the corpus:
Picture Brides (Score: 0.5516)
A Korean in Paris (Score: 0.5418)
Golden Arrow (Score: 0.5254)
Desert Legion (Score: 0.5107)
Hotel Chelsea (Score: 0.5045)


Let us calculate the values of @Recall and @MRR now. Recall@k = true positives@k + true negatives@k

Documentaries showcasing indigenous pepoles survival and daily life in arctic regions The top 5 results for this query are: picture Brides, A korean in paris, Golden Arrow, Desert Legion, Hotel Chelsea out of 5
recall@1 : 1/(1+2)= 0.33

The metric is useful when we our system to return the best relevant item and want that item to be at higher position.
The top 5 results for this query are :  picture Brides, A korean in paris, Golden Arrow, Desert Legion, Hotel Chelsea
For this query, the reciporcal rank is 1/1 and MRR=1 (as the first correct item is at position)

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch

# Load the sentence transformer model
model = SentenceTransformer('nq-distilbert-base-v1')

# Load your data here
# For the sake of this example, let's assume it's already loaded in a variable `data`
data = pd.read_csv('/content/processed_movies.csv')

# Combine the Title and Plot for indexing
corpus = [f"{row['Title']} {row['Plot']}" for _, row in data.iterrows()]

# Encode the corpus using the model
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

# Function to search the corpus
def search(query: str, top_k: int = 5):
    # Encode the query
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Compute the similarities between the query and the corpus
    similarities = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]

    # Find the top_k most similar entries
    top_results = torch.topk(similarities, k=top_k)

    print("Query:", query)
    print("\nTop 5 most similar movies in the corpus:")
    for score, idx in zip(top_results[0], top_results[1]):
        print(data.iloc[idx.item()]['Title'], "(Score: {:.4f})".format(score))

# Get a search query from the user
user_query = input("Enter your movie plot query: ")
search(user_query)

Enter your movie plot query: "Comedy film, office disguises, boss's daughter, elopement
Query: "Comedy film, office disguises, boss's daughter, elopement

Top 5 most similar movies in the corpus:
Lal Dupatta Malmal Ka (Score: 0.4546)
Ami Aar Amar Girlfriends (Score: 0.4540)
The Boy Friend (Score: 0.4454)
Lock Up Your Daughters! (Score: 0.4448)
Interlude (Score: 0.4402)


Let us calculate the values of @Recall and @MRR now. Recall@k = true positives@k + true negatives@k

Documentaries showcasing indigenous pepoles survival and daily life in arctic regions The top 5 results for this query are: Lal dupatta Malmal ka, ami aar amar girlfriend, the boy friend, lockup your duaghters!, interlude out of 5
recall@1 - 1/(1+2)= 0.33

The metric is useful when we our system to return the best relevant item and want that item to be at higher position.
The top 5 results for this query are : Lal dupatta Malmal ka, ami aar amar girlfriend, the boy friend, lockup your duaghters!, interlude
For this query, the reciporcal rank is 1/1 and MRR=1 (as the first correct item is at position)

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch

# Load the sentence transformer model
model = SentenceTransformer('nq-distilbert-base-v1')

# Load your data here
# For the sake of this example, let's assume it's already loaded in a variable `data`
data = pd.read_csv('/content/processed_movies.csv')

# Combine the Title and Plot for indexing
corpus = [f"{row['Title']} {row['Plot']}" for _, row in data.iterrows()]

# Encode the corpus using the model
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

# Function to search the corpus
def search(query: str, top_k: int = 5):
    # Encode the query
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Compute the similarities between the query and the corpus
    similarities = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]

    # Find the top_k most similar entries
    top_results = torch.topk(similarities, k=top_k)

    print("Query:", query)
    print("\nTop 5 most similar movies in the corpus:")
    for score, idx in zip(top_results[0], top_results[1]):
        print(data.iloc[idx.item()]['Title'], "(Score: {:.4f})".format(score))

# Get a search query from the user
user_query = input("Enter your movie plot query: ")
search(user_query)

Enter your movie plot query: "Lost film, Cleopatra charms Caesar, plots world rule, treasures from mummy, revels with Antony, tragic end with serpent in Alexandria
Query: "Lost film, Cleopatra charms Caesar, plots world rule, treasures from mummy, revels with Antony, tragic end with serpent in Alexandria

Top 5 most similar movies in the corpus:
Cleopatra (Score: 0.5163)
Caesar and Cleopatra (Score: 0.4826)
Love's Labour's Lost (Score: 0.4508)
Lost and Found (Score: 0.4473)
Royal Treasure (Score: 0.4423)


Let us calculate the values of @Recall and @MRR now. Recall@k = true positives@k + true negatives@k

Documentaries showcasing indigenous pepoles survival and daily life in arctic regions The top 5 results for this query are:cleopatra, caesar and cleopatra, loves labours lost, lost and found, royal treasure
Reacall@1- 1/(1+2) = 0.33

The metric is useful when we our system to return the best relevant item and want that item to be at higher position.
The top 5 results for this query are : cleopatra, caesar and cleopatra, loves labours lost, lost and found, royal treasure
For this query, the reciporcal rank is 1/1 and MRR=1 (as the first correct item is at position)

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch

# Load the sentence transformer model
model = SentenceTransformer('nq-distilbert-base-v1')

# Load your data here
# For the sake of this example, let's assume it's already loaded in a variable `data`
data = pd.read_csv('/content/processed_movies.csv')

# Combine the Title and Plot for indexing
corpus = [f"{row['Title']} {row['Plot']}" for _, row in data.iterrows()]

# Encode the corpus using the model
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

# Function to search the corpus
def search(query: str, top_k: int = 5):
    # Encode the query
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Compute the similarities between the query and the corpus
    similarities = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]

    # Find the top_k most similar entries
    top_results = torch.topk(similarities, k=top_k)

    print("Query:", query)
    print("\nTop 5 most similar movies in the corpus:")
    for score, idx in zip(top_results[0], top_results[1]):
        print(data.iloc[idx.item()]['Title'], "(Score: {:.4f})".format(score))

# Get a search query from the user
user_query = input("Enter your movie plot query: ")
search(user_query)

Enter your movie plot query: Denis Gage Deane-Tanner
Query: Denis Gage Deane-Tanner

Top 5 most similar movies in the corpus:
The Baxter (Score: 0.4854)
Meet Simon Cherry (Score: 0.4470)
Wedding Rehearsal (Score: 0.4275)
Boys Are Back, TheThe Boys Are Back (Score: 0.4087)
The Running Man (Score: 0.4085)


Let us calculate the values of @Recall and @MRR now. Recall@k = true positives@k + true negatives@k

Documentaries showcasing indigenous pepoles survival and daily life in arctic regions The top 5 results for this query are:The Baxter, meet simon cherry, wedding reharsal,boys are back, the running man
Recall@1- 1/(1+1)= 0.5

The metric is useful when we our system to return the best relevant item and want that item to be at higher position.
The top 5 results for this query are : The Baxter, meet simon cherry, wedding reharsal,boys are back, the running man
For this query, the reciporcal rank is 1/1 and MRR=1 (as the first correct item is at position)